In [132]:
import scipy.io.wavfile as wav
import numpy as np

def read_wave_file(file_path) -> tuple[int, np.ndarray]:
    rate, data = wav.read(file_path)
    if data.ndim == 2:
        data = data[:, 0]
    # convert to float
    if data.dtype == 'int16':
        data = data / 32768.0
    elif data.dtype == 'float32':
        pass
    else:
        raise ValueError("Unknown data type")
    return rate, data

def save_wave_file(file_path, rate, data):
    # Ensure data is in the correct format
    if data.dtype != 'float32':
        data = data.astype('float32')

    # Convert float data to int16 for saving
    data_int16 = (data * 32767).astype('int16')

    # Write the data to a wav file
    wav.write(file_path, rate, data_int16)

# plot the audio data
# use popup window
%matplotlib qt
import matplotlib.pyplot as plt

import sounddevice as sd

def plot_audio(data, rate, seconds=1):
    # If seconds is None, plot the entire audio data
    if seconds is None:
        num_samples = len(data)
        seconds = num_samples / rate
    else:
        num_samples = min(int(seconds * rate), len(data))
        seconds = min(seconds, len(data) / rate)

    # Create a time axis based on the sample rate and the specified duration
    time = np.linspace(0., seconds, num_samples)

    # Plot the audio data for the specified duration
    plt.figure()
    plt.plot(time, data[:num_samples])
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title(f'Audio Data (First {seconds} seconds)' if seconds is not None else 'Audio Data (Full)')
    plt.show()

def plot_fft(data, rate, seconds=1, freq_range=(0, 24000)):
    # If seconds is None, perform FFT on the entire audio data
    if seconds is None:
        num_samples = len(data)
    else:
        num_samples = min(int(seconds * rate), len(data))
        seconds = min(seconds, len(data) / rate)
    # assert num_samples % 2 == 0
    if num_samples % 2 != 0:
        num_samples -= 1
    # Perform FFT on the audio data
    freqs = np.fft.fft(data[:num_samples])
    freqs = np.abs(freqs) / num_samples
    freqs = freqs[:num_samples // 2]
    plt.figure()
    plt.plot(np.linspace(0., rate / 2, num_samples // 2), freqs)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.title(f'FFT of Audio Data (First {seconds} seconds)' if seconds is not None else 'FFT of Audio Data (Full)')
    plt.xlim(*freq_range)
    plt.show()

def plot_wav_file(file_path, seconds=1):
    rate, data = read_wave_file(file_path)
    plot_audio(data, rate, seconds)
    plot_fft(data, rate, seconds)

from scipy.signal import butter, filtfilt

def low_pass_filter(data, rate, freq):
    nyquist = 0.5 * rate
    normal_cutoff = freq / nyquist
    b, a = butter(5, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

def play_audio(audio: np.array, rate=48000):
    save_wave_file("../build/temp.wav", rate, audio)
    sd.play(audio, rate)
    sd.wait()

def audio_by_function(func: callable, rate=48000, duration=1):
    timepoints = np.linspace(0, duration, int(rate * duration))
    return func(timepoints)

def play_audio_by_function(func: callable, rate=48000, duration=1):
    timepoints = np.linspace(0, duration, int(rate * duration))
    audio = audio_by_function(func, rate, duration)
    play_audio(audio, rate)

def sine_wave(freq_hz, phase=0):
    return lambda time_in_second: np.sin(2 * np.pi * freq_hz * time_in_second + phase)

def cosine_wave(freq_hz, phase=0):
    return lambda time_in_second: np.sin(2 * np.pi * freq_hz * time_in_second + phase)

sine_440hz = sine_wave(440)
sine_880hz = sine_wave(880)

def square_wave(freq):
    return lambda time_in_second: np.sign(np.sin(2 * np.pi * freq * time_in_second))

def cosine_440hz(time_in_second):
    return np.cos(2 * np.pi * 440 * time_in_second)

def multiply_amplitude(amplitude: float, func: callable):
    def wrapper(*args, **kwargs):
        return amplitude * func(*args, **kwargs)
    return wrapper

def sine_wave_samples(freq, cycle, phase, amplitude, rate=48000):
    duration = cycle / freq
    timepoints = np.linspace(0, duration, int(rate * duration))
    return multiply_amplitude(amplitude, sine_wave(freq, phase))(timepoints)

def play_sine_waves(sine_waves: tuple[int, float, float], rate=48000):
    audio = np.zeros(0)
    for freq, cycle, amplitude in sine_waves:
        sine_wave_audio = sine_wave_samples(freq, cycle, 0, amplitude, rate)
        audio = np.concatenate((audio, sine_wave_audio))
    play_audio(audio, rate)

# play_sine_waves([(4800, 4, 0.5), (4800, 4, 1), (2400, 4, 1)] * 500)

# play_audio_by_function(sine_wave(440), duration=10)
# play_audio_by_function(sine_wave(4800), duration=3)
# play_audio_by_function(lambda t: (sine_wave(440)(t) + sine_wave()(t))/2, duration=2)

def play_and_record(data, rate, discard=True):
    save_wave_file("../build/temp.wav", rate, data)

    cmd = "../build/supersonic/play_and_record -f ../build/temp.wav -i 'UGREEN CM564 USB Audio  Mono:capture_MONO' -o 'USB2.0 Device Analog Stereo:playback_FL'"
    # cmd = "..\\out\\build\\x64-debug\\supersonic\\play_and_record.exe -f ../build/temp.wav"
    import subprocess
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    p.wait()
    if p.returncode != 0:
        print(p.stdout.read())
        print(p.stderr.read())
        raise ValueError("play_and_record failed")
    
    rate, record_wave = read_wave_file("raw_input.wav")
    rate, play_wave = read_wave_file("raw_output.wav")
    if discard:
        # discard first 0.5s
        record_wave = record_wave[int(0.5 * rate):]
        play_wave = play_wave[int(0.5 * rate):]
    return record_wave, play_wave, rate

# try play a chirp

# chirp_frequencies1 = np.linspace(5000, 7500, 20)
# chirp_frequencies2 = np.linspace(7500, 10000, 20)

# def gen_chirp(frequencies):
#     rate = 48000
#     duration = 1 / 100
#     timepoints = np.linspace(0, duration, int(rate * duration) + 1)
#     chirp = np.zeros(len(timepoints))
#     for freq in frequencies:
#         # print(freq)
#         chirp += sine_wave(freq)(timepoints)
#     chirp /= len(frequencies)
#     return chirp

# chirp1 = gen_chirp(chirp_frequencies1)
# chirp2 = gen_chirp(chirp_frequencies2)

def gen_chirp(f_0, c, duration, rate):
    f_1 = c * duration + f_0
    print(f_0, f_1)

    def phi(t):
        return 2 * np.pi * (c / 2 * t * t + f_0 * t)

    times = np.linspace(0, duration, int(rate * duration), endpoint=False)
    chirp = np.sin(phi(times))
    chirp_rev = -np.flip(chirp)
    chirp = np.concatenate((chirp, chirp_rev))
    return chirp
chirp1 = gen_chirp(3000, 400000, 0.01, 48000)
chirp2 = gen_chirp(7000, 400000, 0.01, 48000)

def locate_chirp(data, chirp):
    corr = np.correlate(data, chirp, mode='full')
    loc = np.argmax(corr)
    return loc

def play_and_record_precise(data, rate):
    # get sine wave with 440Hz for 0.2s
    sine_wav = sine_wave_samples(440, 1 * 440, 0, 1, rate)
    audio = np.concatenate((sine_wav, chirp1, data, chirp2, np.zeros(int(0.5*rate))))

    record_wave, play_wave, rate = play_and_record(audio, rate, discard=False)

    magic_correction = 0

    chirp1_loc = locate_chirp(record_wave, chirp1) + magic_correction
    chirp2_loc = locate_chirp(record_wave, chirp2) + magic_correction
    chirp2_loc = min(chirp2_loc, len(record_wave))

    start = chirp1_loc + 1
    end = chirp2_loc - len(chirp2) + 1

    return record_wave[start:end], rate

def play_and_record_precise2(data, rate):
    # get sine wave with 440Hz for 0.2s
    sine_wav = sine_wave_samples(440, 1 * 440, 0, 1, rate)
    audio = np.concatenate((sine_wav, chirp1, data, chirp2, np.zeros(int(0.5*rate))))

    record_wave, play_wave, rate = play_and_record(audio, rate, discard=False)

    magic_correction = 0

    chirp1_loc = locate_chirp(record_wave, chirp1) + magic_correction
    chirp2_loc = locate_chirp(record_wave, chirp2) + magic_correction
    chirp2_loc = min(chirp2_loc, len(record_wave))

    start = chirp1_loc + 1
    end = chirp2_loc - len(chirp2) + 1
    print(start, end)

    return record_wave[start:end], rate, record_wave

3000 7000.0
7000 11000.0


In [41]:
rate = 48000
freq = 10
cycles = 1
amplitude = 1
window_size = int(rate / freq * cycles)

one = sine_wave_samples(freq, cycles, 0, amplitude)

def plot_modulation(one, zero, title_prefix):
    # zero_shift = zero * one
    # zero_smooth = np.convolve(zero_shift, np.ones(window_size) / window_size, mode='full')
    # fig, axs = plt.subplots(2, 2)
    # axs[0, 0].plot(one)
    # axs[0, 0].set_title(f"{title_prefix} One")
    # axs[0, 0].set_ylim([-1, 1])
    # axs[0, 1].plot(zero)
    # axs[0, 1].set_title(f"{title_prefix} Zero")
    # axs[0, 1].set_ylim([-1, 1])
    # axs[1, 0].plot(zero_shift)
    # axs[1, 0].set_title(f"{title_prefix} Zero Shift")
    # axs[1, 0].set_ylim([-1, 1])
    # axs[1, 1].plot(zero_smooth)
    # axs[1, 1].set_title(f"{title_prefix} Zero Smooth")
    # axs[1, 1].set_ylim([-1, 1])
    # axs[1, 1].axvline(x=window_size, color='r', linestyle='--')
    # axs[1, 1].annotate(f'Result: {zero_smooth[window_size]:.2e}', xy=(0.5, 0.9), xycoords='axes fraction', fontsize=10, ha='center', va='center', bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white'))
    # plt.show()
    one_shift = one * one
    one_smooth = np.correlate(one_shift, np.ones(window_size) / window_size, mode='full')
    zero_shift = zero * one
    zero_smooth = np.correlate(zero_shift, np.ones(window_size) / window_size, mode='full')
    fig, axs = plt.subplots(2, 3)
    axs[0, 0].plot(one)
    axs[0, 0].set_title(f"{title_prefix} One")
    axs[0, 0].set_ylim([-1, 1])
    axs[0, 1].plot(one_shift)
    axs[0, 1].set_title(f"{title_prefix} One Shift")
    axs[0, 1].set_ylim([-1, 1])
    axs[0, 2].plot(one_smooth)
    axs[0, 2].set_title(f"{title_prefix} One Smooth")
    axs[0, 2].set_ylim([-1, 1])
    axs[0, 2].axvline(x=window_size, color='r', linestyle='--')
    axs[0, 2].annotate(f'Result: {one_smooth[window_size]:.2e}', xy=(0.5, 0.9), xycoords='axes fraction', fontsize=10, ha='center', va='center', bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white'))
    axs[1, 0].plot(zero)
    axs[1, 0].set_title(f"{title_prefix} Zero")
    axs[1, 0].set_ylim([-1, 1])
    axs[1, 1].plot(zero_shift)
    axs[1, 1].set_title(f"{title_prefix} Zero Shift")
    axs[1, 1].set_ylim([-1, 1])
    axs[1, 2].plot(zero_smooth)
    axs[1, 2].set_title(f"{title_prefix} Zero Smooth")
    axs[1, 2].set_ylim([-1, 1])
    axs[1, 2].axvline(x=window_size, color='r', linestyle='--')
    axs[1, 2].annotate(f'Result: {zero_smooth[window_size]:.2e}', xy=(0.5, 0.9), xycoords='axes fraction', fontsize=10, ha='center', va='center', bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white'))
    plt.show()

# ASK, Amplitude Shift Keying
zero_ask = sine_wave_samples(freq, cycles, 0, 0.5)
plot_modulation(one, zero_ask, "ASK")
# one = 0.5
# zero = 0.25

# FSK, Frequency Shift Keying
zero_fsk = sine_wave_samples(freq * 2, cycles * 2, 0, 1)
plot_modulation(one, zero_fsk, "FSK")
# one = 0.5
# zero = 0

# PSK, Phase Shift Keying
zero_psk = sine_wave_samples(freq, cycles, np.pi, 1)
plot_modulation(one, zero_psk, "PSK pi")
# one = 0.5
# zero = -0.5

# PSK, Phase Shift Keying
zero_psk = sine_wave_samples(freq, cycles, np.pi/2, 1)
plot_modulation(one, zero_psk, "PSK pi/2")
# one = 0.5
# zero = 0

The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x650b28b774a0
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x650b290de790
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x650b291340

In [82]:
rate = 10000
freq = 10
cycles = 1
amplitude = 1
window_size = int(rate / freq * cycles)

duration = cycles / freq
timepoints = np.linspace(0, duration, int(rate * duration))

one = np.sin(2 * np.pi * freq * timepoints)
zero = np.cos(2 * np.pi * 2 * freq * timepoints)
# one = np.ones(len(timepoints))
carrier = np.cos(2 * np.pi * 200 * timepoints)

one_passband = one * carrier
zero_passband = zero * carrier

one_decode = one_passband * carrier

plot_fft(one_decode, rate)

# plt.plot(one_passband)
# plt.plot(one_decode)
# (one_decode, rate)
print(np.dot(one_decode, np.sin(2 * np.pi * freq * timepoints)))
print(np.dot(one_decode, np.cos(2 * np.pi * freq * timepoints)))

print(np.dot(one_decode, np.sin(2 * np.pi * 2 * freq * timepoints)))
print(np.dot(one_decode, np.cos(2 * np.pi * 2 * freq * timepoints)))

249.74999999999986
-1.5727643521394e-13
2.8395679091495346e-13
-1.2243277947546984e-13


The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x58dae9a30bd0


In [185]:
rate = 20

freq = 10
cycles = 20
amplitude = 1
window_size = int(rate / freq * cycles)

duration = cycles / freq
x = np.linspace(0, duration, int(rate * duration))
x_more = np.linspace(0, duration, int(rate * duration * 10))

wave = sine_wave(freq)(x)

plt.plot(x, wave)
# plt.plot(x_more, sine_wave(freq)(x_more))

# plot_fft(wave, rate)

In [186]:
t = np.linspace(0, 1, 48000)
# wave = sine_wave(200)(t) * sine_wave(1000)(t)
wave = sine_wave(200)(t) * sine_wave(200, np.pi / 2)(t)
plot_fft(wave, 48000)

In [187]:
plot_wav_file("../build/temp.wav", seconds=None)

In [352]:
rate = 48000
duration = 5

none_wave = np.zeros(int(rate * duration))

noise_wave, _, rate = play_and_record(none_wave, rate)

In [353]:
# calculate noise power
noise_power = np.sum(noise_wave ** 2) / len(noise_wave) * rate
print(f"noise power: {noise_power}")

noise power: 0.3337700366973877


In [354]:
plot_fft(noise_wave, rate)

In [355]:
class FrequencyData:
    def __init__(self, freq, wave, signal, noise):
        self.freq = freq
        self.wave = wave
        self.s_div_n = signal / noise
        print(f"freq={freq} S/N={self.s_div_n}")
freq_data_map = {}

In [357]:
def test_freq_SNR(freq):
    rate = 48000
    duration = 1.5
    cycles = duration * freq
    phase = 0
    amplitude = 1

    # print(f"Freq = {freq}, Period (s) = {1 / freq}")

    wave = sine_wave_samples(freq, cycles, phase, amplitude, rate)
    record_wave, play_wave, rate = play_and_record(wave, rate)

    # calculate signal power
    signal_noise_power = np.sum(record_wave ** 2) / len(record_wave) * rate
    signal_power = signal_noise_power - noise_power
    # print(f"freq={freq} signal power: {signal_power}")

    # calculate SNR
    # print(f"freq={freq} S/N={signal_power / noise_power}")
    # print(f"freq={freq} SNR={10 * np.log10(signal_power / noise_power)}")

    freq_data_map[freq] = FrequencyData(freq, record_wave, signal_power, noise_power)

for freq in range(200, 1000, 100):
    test_freq_SNR(freq)

for freq in range(1000, 10000, 500):
    test_freq_SNR(freq)

for freq in range(10000, 15000, 500):
    test_freq_SNR(freq)

for freq in range(15000, 20000, 500):
    test_freq_SNR(freq)

# for freq in [100, 200, 400, 800, 1600, 3200, 6400, 12800]:
#     test_freq_SNR(freq)

freq=200 S/N=6180.28564453125
freq=300 S/N=10547.876953125
freq=400 S/N=6437.01708984375
freq=500 S/N=2790.4609375
freq=600 S/N=20332.91015625
freq=700 S/N=23913.248046875
freq=800 S/N=2184.93408203125
freq=900 S/N=8733.72265625
freq=1000 S/N=9756.5615234375
freq=1500 S/N=33645.984375
freq=2000 S/N=33775.51953125
freq=2500 S/N=41350.671875
freq=3000 S/N=5711.15185546875
freq=3500 S/N=2145.159912109375
freq=4000 S/N=10867.0
freq=4500 S/N=21287.65625
freq=5000 S/N=3625.360595703125
freq=5500 S/N=506.87750244140625
freq=6000 S/N=9915.4169921875
freq=6500 S/N=1332.413818359375
freq=7000 S/N=2423.74658203125
freq=7500 S/N=833.5259399414062
freq=8000 S/N=4672.8427734375
freq=8500 S/N=8913.705078125
freq=9000 S/N=9868.794921875
freq=9500 S/N=5061.52880859375
freq=10000 S/N=696.3435668945312
freq=10500 S/N=1472.89501953125
freq=11000 S/N=1352.660400390625
freq=11500 S/N=798.1116333007812
freq=12000 S/N=100.519287109375
freq=12500 S/N=949.373779296875
freq=13000 S/N=1223.51904296875
freq=13500 

In [368]:
# plot_fft(freq_data_map[13000].wave, rate)
# plot freq and S/N
freqs = sorted(list(freq_data_map.keys()))
s_div_n = [freq_data_map[freq].s_div_n for freq in freqs]
# plt.plot(freqs, s_div_n)
plt.title("Freq - log2(1 + S/N)")
plt.plot(freqs, np.log2(1 + np.array(s_div_n)))

In [132]:
# plot_audio(freq_data_map[13500].wave, rate)
# plot_fft(freq_data_map[13500].wave, rate)
# plot_audio(sine_wave(13500)(np.linspace(0, 0.2, 48000)), 48000)

x = np.linspace(0, 0.01, int(0.01*48000))
x_fine = np.linspace(0, 0.01, 10000)
y = sine_wave(13500)(x)
y_fine = sine_wave(13500)(x_fine)
# plot graph and point
plt.plot(x, y)
plt.plot(x_fine, y_fine)
plt.scatter(x, y)
plt.show()

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x5c0a3be4d130


In [114]:
B = 13500 - 8500
C = B * np.log2(1 + freq_data_map[8500].s_div_n)
print(f"Channel Capacity: {C} bps, {C/1000} kbps")

Channel Capacity: 22573.1796875 bps, 22.573179244995117 kbps


In [369]:
# try play square wave
rate = 48000
freq = 500
duration = 1
timepoints = np.linspace(0, duration, int(rate * duration))
wave = square_wave(freq)(timepoints)
record_wave, play_wave, rate = play_and_record(wave, rate)

In [375]:
# try play sine wave
rate = 48000
freq = 500
duration = 1
timepoints = np.linspace(0, duration, int(rate * duration))
wave = sine_wave(freq)(timepoints)
record_wave, play_wave, rate = play_and_record(wave, rate)

In [161]:
# try play triangle wave
rate = 48000
freq = 500
duration = 1
timepoints = np.linspace(0, duration, int(rate * duration))
def triangle_wave(freq):
    return lambda time_in_second: np.abs(2 * (time_in_second * freq - np.floor(time_in_second * freq + 0.5)))
wave = triangle_wave(freq)(timepoints)
record_wave, play_wave, rate = play_and_record(wave, rate)

In [127]:
# try play a chord
rate = 48000
duration = 5
timepoints = np.linspace(0, duration, int(rate * duration) + 1)
wave = np.zeros(len(timepoints))
count = 0
frequencies = [261.63, 327.04, 392.45]

for freq in frequencies:
    # print(freq)
    count += 1
    wave += sine_wave(freq)(timepoints)
wave /= count
record_wave, play_wave, rate = play_and_record(wave, rate)

In [128]:
rate, wave = read_wave_file("../test-audio.wav")
wave = wave[int(0.7*rate):int(1.7*rate)]
# sd.play(wave, rate)
record, rate = play_and_record_precise(wave, rate)

print(len(wave), len(record))

48000 47724


In [144]:
rate, wave = read_wave_file("../test-audio.wav")
seconds = 2
start_point = int(0.75 * rate)
# carrier_wave = wave[start_point:start_point+int(rate * seconds)]
carrier_wave = sine_wave(440)(np.linspace(0, seconds, int(rate * seconds), endpoint=False))

# encode binary data to wave with frequency freq
freq = 100
bin1 = np.random.randint(0, 2, size=int(freq * seconds))
samples_per_symbol = int(rate / freq)
fsk_shift_freq = 440

fsk_waves = []
for i in range(len(bin1)):
    bit = bin1[i]
    start = i * samples_per_symbol
    end = (i + 1) * samples_per_symbol
    one = carrier_wave[start:end]
    # shift freq by fsk_shift_freq Hz
    zero = carrier_wave[start:end] * cosine_wave(fsk_shift_freq)(np.linspace(0, 1, samples_per_symbol, endpoint=False))
    fsk_waves.append(one if bit == 1 else zero)
fsk_wave1 = np.concatenate(fsk_waves)

modulated_wave = fsk_wave1

In [145]:
# receive_wave = modulated_wave
receive_wave, rate = play_and_record_precise(modulated_wave, rate)
# receive_wave, rate = play_and_record_precise(carrier_wave, rate)

In [147]:
print(len(receive_wave), len(modulated_wave))
if len(receive_wave) > len(modulated_wave):
    receive_psk_wave = receive_wave[:len(modulated_wave)]
else:
    receive_psk_wave = np.concatenate((receive_wave, np.zeros(len(modulated_wave) - len(receive_wave))))

receive_bits = np.zeros(len(bin1))
for i in range(len(bin1)):
    start = i * samples_per_symbol
    end = (i + 1) * samples_per_symbol
    one = carrier_wave[start:end]
    zero = carrier_wave[start:end] * cosine_wave(fsk_shift_freq)(np.linspace(0, 1, samples_per_symbol, endpoint=False))

    one_corr = np.dot(one, receive_psk_wave[start:end])
    zero_corr = np.dot(zero, receive_psk_wave[start:end])
   
    print(one_corr, zero_corr)
    bit = 1 if np.abs(one_corr) > np.abs(zero_corr) else 0
    receive_bits[i] = bit

print(bin1)

similarity = np.sum(receive_bits == bin1) / len(bin1)
# print(f"Binary data: {binary_data}")
print(f"Similarity: {similarity}")

97004 96000
Similarity: 0.45


In [117]:
data_len = 50
bin1 = np.random.randint(0, 2, size=data_len)
bin2 = np.random.randint(0, 2, size=data_len)

def fsk_modulation(binary_data, rate, symbol_freq, fsk_freq):
    symbol_samples = int(rate / symbol_freq)
    symbol_time = 1 / symbol_freq

    print(f"Min FSK freq: {symbol_freq}")
    assert fsk_freq >= symbol_freq

    one = sine_wave(fsk_freq)(np.linspace(0, symbol_time, symbol_samples, endpoint=False))
    zero = sine_wave(2*fsk_freq)(np.linspace(0, symbol_time, symbol_samples, endpoint=False))

    fsk_wave = np.concatenate([one if bit == 1 else zero for bit in binary_data])
    return fsk_wave

def fsk_demodulation(expected_bits, receive_wave, rate, symbol_freq, fsk_freq):
    symbol_samples = int(rate / symbol_freq)
    symbol_time = 1 / symbol_freq

    one = sine_wave(fsk_freq)(np.linspace(0, symbol_time, symbol_samples, endpoint=False))
    zero = sine_wave(2*fsk_freq)(np.linspace(0, symbol_time, symbol_samples, endpoint=False))

    receive_fsk_wave = receive_wave

    num_recv = 0
    receive_bits = np.zeros(expected_bits)
    start = 0
    wave_lens = []
    while num_recv < expected_bits:
        end = start + int(symbol_samples*1.2)
        wave = receive_fsk_wave[start:end]
        wave = np.concatenate((wave, np.zeros(end - start - len(wave))))

        # plot_audio(wave, rate, seconds=None)
        # plot_fft(wave, rate, seconds=None, freq_range=(500, 2500))

        one_corr_arr = np.correlate(wave, one, 'full')[:len(wave)]
        zero_corr_arr = np.correlate(wave, zero, 'full')[:len(wave)]
        one_corr = np.max(one_corr_arr)
        zero_corr = np.max(zero_corr_arr)
        bit = 1 if one_corr > zero_corr else 0

        wave_len = np.argmax(one_corr_arr) if bit == 1 else np.argmax(zero_corr_arr)
        start += wave_len
        wave_lens.append(wave_len)

        print(f"bit={bit}, truth={bin1[num_recv]}, wave_len={wave_len} one_corr = {one_corr}, zero_corr = {zero_corr}")

        receive_bits[num_recv] = bit
        num_recv += 1
    return receive_bits

rate = 48000

symbol_freq1 = 1000
fsk_freq1 = symbol_freq1
fsk_wave1 = fsk_modulation(bin1, rate, symbol_freq1, fsk_freq1)

# symbol_freq2 = 1500
# fsk_freq2 = symbol_freq2
# fsk_wave2 = fsk_modulation(bin2, rate, symbol_freq2, fsk_freq2)

# fsk_wave = np.zeros(max(len(fsk_wave1), len(fsk_wave2)))
# fsk_wave[:len(fsk_wave1)] += fsk_wave1
# fsk_wave[:len(fsk_wave2)] += fsk_wave2
# fsk_wave /= 2

fsk_wave = fsk_wave1

record_wave, rate = play_and_record_precise(fsk_wave, rate)
print(len(record_wave), len(fsk_wave))

receive_bits = fsk_demodulation(len(bin1), record_wave.copy(), rate, symbol_freq1, fsk_freq1)

similarity = np.sum(receive_bits == bin1) / len(bin1)
print(f"Similarity: {similarity}")


transfer_time = len(fsk_wave) / rate
bits = len(bin1)
bps = bits / transfer_time
print(f"Transfer Time: {transfer_time} s, {bps} bps, {bps/1000} kbps")

Min FSK freq: 1000
2450 2400
bit=1, truth=1, wave_len=56 one_corr = 0.5796708416119744, zero_corr = 0.18639372630364856
bit=0, truth=0, wave_len=45 one_corr = 0.23382373333045506, zero_corr = 0.9449798879965285
bit=0, truth=0, wave_len=48 one_corr = 0.3322032630853176, zero_corr = 1.0369042071720256
bit=0, truth=0, wave_len=49 one_corr = 0.24895658111369584, zero_corr = 0.9850747331348807
bit=0, truth=0, wave_len=48 one_corr = 0.274384364958881, zero_corr = 0.9394127411348869
bit=1, truth=1, wave_len=54 one_corr = 0.5702854945850174, zero_corr = 0.16052222188778456
bit=0, truth=0, wave_len=44 one_corr = 0.21013670411121, zero_corr = 0.9500805925483123
bit=0, truth=0, wave_len=48 one_corr = 0.2815951204276254, zero_corr = 1.0110102110160355
bit=0, truth=0, wave_len=49 one_corr = 0.23218137123919552, zero_corr = 0.9952529673962611
bit=0, truth=0, wave_len=48 one_corr = 0.28711796151720653, zero_corr = 0.9376595342050116
bit=1, truth=1, wave_len=56 one_corr = 0.5935757496145704, zero_corr

In [121]:
# plot_audio(fsk_wave, rate)
# plot_audio(record_wave, rate)
plt.plot(fsk_wave * 0.03)
plt.plot(record_wave)
for i in range(len(bin1)):
    plt.axvline(x=i*rate/symbol_freq1, color='r', linestyle='--')

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x5a5294ae5020


In [85]:
plot_fft(record_wave, rate, seconds=None, freq_range=(0, 4500))

The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x5a5295142180


In [106]:
# plot_audio(receive_wave, rate, seconds=None)
plt.plot(receive_wave)

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x650b28899080


In [137]:
data_len = 10
# binary_data = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])
bin1 = np.random.randint(0, 2, size=data_len)
# binary_data = np.ones(data_len)
# binary_data = np.zeros(data_len)
bin1

# PSK modulation
rate = 48000

symbol_freq1 = 200
symbol_samples = int(rate / symbol_freq1) 
symbol_time = 1 / symbol_freq1
symbol_times = np.linspace(0, symbol_time, symbol_samples, endpoint=False)

psk_freq = 400

transfer_time = len(bin1) * symbol_time
# carrier_freq = 2000
# def carrier_wave_f(samples):
#     return cosine_wave(carrier_freq)(np.linspace(0, samples / rate, samples, endpoint=False))
# carrier_wave = carrier_wave_f(len(bin1) * symbol_samples)

print(f"Min PSK freq: {symbol_freq1}")
assert psk_freq >= symbol_freq1

one = sine_wave(psk_freq)(symbol_times)
zero = sine_wave(psk_freq, np.pi)(symbol_times)

psk_wave = np.concatenate([one if bit == 1 else zero for bit in bin1])

# play_wave = psk_wave * carrier_wave

record_wave, rate, raw_record_wave = play_and_record_precise2(psk_wave, rate)
print(len(record_wave), len(psk_wave))


# # plot_fft(record_wave, rate, seconds=None)

# # decode fsk_wave
# # receive_wave = record_wave.copy()
# # if len(receive_wave) > len(fsk_wave):
# #     receive_fsk_wave = receive_wave[:len(fsk_wave)]
# # else:
# #     receive_fsk_wave = np.concatenate((receive_wave, np.zeros(len(fsk_wave) - len(receive_wave))))
# # receive_bits = np.zeros(len(binary_data))
# # for i in range(len(binary_data)):
# #     start = i * symbol_samples
# #     end = (i + 1) * symbol_samples
# #     one_corr = np.dot(one, receive_fsk_wave[start:end])
# #     zero_corr = np.dot(zero, receive_fsk_wave[start:end])
# #     bit = 1 if np.abs(one_corr) > np.abs(zero_corr) else 0
# #     receive_bits[i] = bit
# # similarity = np.sum(receive_bits == binary_data) / len(binary_data)
# # print(f"Similarity: {similarity}")

# # with time sync
# receive_wave = record_wave.copy()
# receive_psk_wave = receive_wave * carrier_wave_f(len(receive_wave))
# receive_psk_wave = low_pass_filter(receive_psk_wave, rate, carrier_freq)
# num_recv = 0
# receive_bits = np.zeros(len(bin1))
# start = 0
# wave_lens = []
# while num_recv < len(bin1):
#     end = start + int(symbol_samples)
#     wave = receive_psk_wave[start:end]
#     wave = np.concatenate((wave, np.zeros(end - start - len(wave))))

#     one_corr_arr = np.correlate(wave, one, 'full')[:len(wave)]
#     sig = one_corr_arr[symbol_samples - 1]
#     bit = 1 if sig > 0 else 0
#     print(f"bit={bit}, truth={bin1[num_recv]}, next_truth={bin1[(num_recv+1)%len(bin1)]} sig={sig}")

#     wave_len = symbol_samples
#     # wave_len = np.argmax(one_corr_arr) if bit == 1 else np.argmax(zero_corr_arr)
#     start += wave_len
#     wave_lens.append(wave_len)
#     receive_bits[num_recv] = bit
#     num_recv += 1

# similarity = np.sum(receive_bits == bin1) / len(bin1)
# print(f"Similarity: {similarity}")
# # print(wave_lens)

Min PSK freq: 200
53371 55791
2420 2400


In [123]:
# plt.plot(psk_wave)
# plt.plot(record_wave)
# plot_fft(psk_wave, rate, seconds=None)

# plot_audio(psk_wave, rate, seconds=None)
# plot_audio(receive_psk_wave, rate, seconds=None)
# plot_fft(psk_wave, rate, seconds=None)

plt.plot(psk_wave * 0.07)
plt.plot(receive_psk_wave)
# # plt.plot(record_wave * np.concatenate([one] * len(binary_data)))
# # plt.plot(np.correlate(record_wave, one, 'full')[:len(record_wave)])
# # plt.plot(record_wave[8:])
for i in range(len(bin1)):
    plt.axvline(x=i*symbol_samples, color='r', linestyle='--')
plt.show()

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x5a52950e3f70


In [129]:
plot_fft(psk_wave, rate, seconds=None)

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x5a5294b32d50


In [131]:
plot_fft(np.array([0] * 100 + [1] * 100), rate, seconds=None)

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x5a5295291590


In [138]:
plot_audio(raw_record_wave, rate, seconds=None)

qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x5a5295153280
